In [89]:
# !pip install openwakeword
# !pip install speechbrain
# !pip install datasets
# !pip install scipy matplotlib

In [90]:
import os
import collections
import numpy as np
from numpy.lib.format import open_memmap
from pathlib import Path

from onnxruntime.transformers.shape_infer_helper import file_path
from tqdm import tqdm
import openwakeword
# import openwakeword.data
import openwakeword.utils
import openwakeword.metrics


import scipy
import datasets
import matplotlib.pyplot as plt
import torch
from torch import nn
import IPython.display as ipd

import datasets
import os
import numpy as np
import scipy.io.wavfile
from tqdm import tqdm

import urllib.request


import warnings

warnings.filterwarnings("ignore")

In [91]:


ds = datasets.load_dataset(
    "speech_commands",
    "v0.02",
    split="test",
    streaming=True
)
ds_iter = iter(ds)

os.makedirs("./data/speech_commands_test", exist_ok=True)
limit = 4890

for i in tqdm(range(limit)):
    output_file = f"./data/speech_commands_test/{i:05d}.wav"

    if os.path.exists(output_file):
        continue

    example = next(ds_iter)
    wav_data = (example["audio"]["array"] * 32767).astype(np.int16) # Convert to 16-bit PCM Format
    scipy.io.wavfile.write(output_file, 16000, wav_data)

100%|██████████| 4890/4890 [00:00<00:00, 16879.25it/s]


# ⇲ Compute Audio Embeddings

Create audio pre-processing object to get openwakeword audio embeddings

In [92]:
model_dir = "./resources/models"
os.makedirs(model_dir, exist_ok=True)

List Model and URL corresponding

In [93]:
models = {
    "embedding_model.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx",
    "embedding_model.tflite": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.tflite",
    "melspectrogram.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.onnx",
    "melspectrogram.tflite": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.tflite"
}

In [94]:
for filename, url in models.items():
    file_path = os.path.join(model_dir, filename)
    if not os.path.exists(file_path):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, file_path)
    else:
        print(f"Found {filename}.")


Found embedding_model.onnx.
Found embedding_model.tflite.
Found melspectrogram.onnx.
Found melspectrogram.tflite.


In [95]:
F = openwakeword.utils.AudioFeatures(
    melspec_model_path=f"{model_dir}/melspectrogram.onnx",
    embedding_model_path=f"{model_dir}/embedding_model.onnx",
    inference_framework="onnx",
)

## ⌀ Negative Clips

Get negative example paths, filtering out clips that are too long or too short

 pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cpu

In [96]:
import openwakeword.data

In [97]:
negative_clips, negative_durations = openwakeword.data.filter_audio_paths(
    [
        "./data/fma_sample",
        "./data/fsd50k_sample",
        "./data/speech_commands_test",
    ],
    min_length_secs=1.0, # minimum clip length in seconds
    max_length_secs=60*30, # maximum clip length in seconds
    duration_method="header" # use the file header to calculate duration
)

print(f"{len(negative_clips)} negative clips after filtering, representing ~{sum(negative_durations)//3600} hours")

200it [00:00, 6419.54it/s]
100%|██████████| 200/200 [00:01<00:00, 107.29it/s]
1000it [00:00, 13571.47it/s]
100%|██████████| 1000/1000 [00:04<00:00, 240.41it/s]
4890it [00:00, 93591.67it/s]
100%|██████████| 4890/4890 [00:08<00:00, 581.03it/s]

5987 negative clips after filtering, representing ~5.0 hours


Use HuggingFace datasets to load files from disk by batches, because:
- Load lazy
- Memmory-Mapping for doesn't load entire in RAM
- map() -> feature extraction, augmentation
- filter() -> remove audio too long/short

In [98]:
audio_dataset = datasets.Dataset.from_dict({
    "audio": negative_clips
})

In [99]:
audio_dataset = audio_dataset.cast_column(
    "audio",
    datasets.Audio(sampling_rate=16000)
)

- Get Audio Embeddings (features) for negative clips and save to .npy file
- Process files by batch and save to Numpy memory mapped file so that
- An array larger than the available system memory can be created

In [100]:
batch_size = 64 # number of files to load, compute features, and write to mmap at a time

In [101]:
clip_size = 3 # The desired window size (in seconds) for the trained openWakeWord model

In [102]:
N_total = int(sum(negative_durations)//clip_size) # Maximum number of rows in mmap file

N_total

6441

In [103]:
n_feature_cols = F.get_embedding_shape(clip_size)

n_feature_cols

(28, 96)

In [104]:
output_file = "negative_features.npy"

In [105]:
output_array_shape = (N_total, n_feature_cols[0], n_feature_cols[1])

output_array_shape

(6441, 28, 96)

Remove .npy before create mmap

In [106]:
for f in [output_file, output_file + ".npy"]:
    if os.path.exists(f):
        os.remove(f)

In [107]:
fp = open_memmap(
    output_file,
    mode="w+",
    dtype=np.float32,
    shape=output_array_shape
)

fp

memmap([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [108]:
row_counter = 0

In [109]:
print("\n===== START PROCESS =====")
print(f"Total rows: {audio_dataset.num_rows}")
print(f"Batch size: {batch_size}")
print(f"Expected batches: {int(np.ceil(audio_dataset.num_rows / batch_size))}")
print("")

for i in tqdm(np.arange(0, audio_dataset.num_rows, batch_size)):
    print(f"\n--- Batch index: {int(i / batch_size)} ---")

    print("Loading audio batch...")
    block = audio_dataset[i:i + batch_size]["audio"]
    print(f"Loaded {len(block)} clips")

    print("Converting to int16 arrays...")
    wav_data = [(j["array"] * 32767).astype(np.int16) for j in block]
    print("Conversion done.")

    print("Stacking clips (pad/trim)...")
    wav_data = openwakeword.data.stack_clips(
        wav_data,
        clip_size=16000 * clip_size
    ).astype(np.int16)
    print("Stack clips done.")

    print("Computing embeddings...")
    features = F.embed_clips(
        x=wav_data,
        batch_size=1024,
        ncpu=8
    )
    print(f"Embedding done. features.shape = {features.shape}")

    print("Saving to mmap...")

    rows_needed = features.shape[0]

    if row_counter + rows_needed > N_total:
        print(f"Final batch: writing {N_total - row_counter} rows then break.")
        fp[row_counter:N_total, :, :] = features[0:N_total - row_counter]
        fp.flush()
        break

    else:
        print(f"Writing rows {row_counter} → {row_counter + rows_needed}")
        fp[row_counter:row_counter + rows_needed, :, :] = features
        row_counter += rows_needed
        fp.flush()


===== START PROCESS =====
Total rows: 5987
Batch size: 64
Expected batches: 94



  0%|          | 0/94 [00:00<?, ?it/s]


--- Batch index: 0 ---
Loading audio batch...
Loaded 64 clips
Converting to int16 arrays...
Conversion done.
Stacking clips (pad/trim)...
Stack clips done.
Computing embeddings...


  1%|          | 1/94 [00:07<11:04,  7.15s/it]

Embedding done. features.shape = (640, 28, 96)
Saving to mmap...
Writing rows 0 → 640

--- Batch index: 1 ---
Loading audio batch...
Loaded 64 clips
Converting to int16 arrays...
Conversion done.
Stacking clips (pad/trim)...
Stack clips done.
Computing embeddings...


...

Embedding done. features.shape = (22, 28, 96)
Saving to mmap...
Writing rows 6380 → 6402

--- Batch index: 89 ---
Loading audio batch...
Loaded 64 clips
Converting to int16 arrays...
Conversion done.
Stacking clips (pad/trim)...
Stack clips done.
Computing embeddings...


 96%|█████████▌| 90/94 [01:24<00:01,  3.18it/s]

Embedding done. features.shape = (22, 28, 96)
Saving to mmap...
Writing rows 6402 → 6424

--- Batch index: 90 ---
Loading audio batch...
Loaded 64 clips
Converting to int16 arrays...
Conversion done.
Stacking clips (pad/trim)...
Stack clips done.
Computing embeddings...


 96%|█████████▌| 90/94 [01:24<00:03,  1.06it/s]

Embedding done. features.shape = (22, 28, 96)
Saving to mmap...
Final batch: writing 17 rows then break.


In [110]:
print("\n===== Trimming mmap file =====")
openwakeword.data.trim_mmap(output_file)
print("===== FINISHED =====\n")


===== Trimming mmap file =====


Trimming empty rows: 7it [00:01,  5.44it/s]                       

===== FINISHED =====



Now we have all of the negative features prepared, and saved to fixed durations clips in Numpy array. For this data, the array is small at ~160MB, but in-practice the memory mapping allows to be very large.

## ✰ Negative Clips